In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import sqlite3 as sql3
import random

random.seed(1000)
np.random.seed(1000)

sns.set(style="darkgrid")
ROOT_PATH="C:/Users/moshe/Documents/projectML/p1"
DATA_PATH = "%s/data/raw" % ROOT_PATH
DB_FILE = "%s/hairsalon.db" % DATA_PATH
def readSQL(query):
    import pandas as pd
    import sqlite3 as sql3
    db = sql3.connect(DB_FILE)
    df = pd.read_sql_query(query, db)
    db.close()
    return(df)

db=sql3.connect(DB_FILE)
from sklearn.feature_selection import SelectFromModel

In [ ]:
query="""select * from finalbooking"""
final=readSQL(query)
for c in final.columns:
    if final[c].nunique() < 14:
        final[c] = final[c].astype('category')
X=final.loc[:,final.columns[1:]]
y=final['noshowflag']

## Univariate Analysis

In [ ]:
import pyMechkar as mechkar
x=final.columns.tolist()
tab1 = mechkar.pyMechkar().Table1(x=x,y="noshowflag",data=final, maxcat=5,miss=3, catmiss=True)

Factorizing... please wait
*********************************************************************************************************************************************************************************
[]
[**********************************************************************************************************************************************************************************
[**********************************************************************************************************************************************************************************


In [ ]:
tab1[tab1['p_value']<0.05]

In [ ]:
vn1 = tab1.loc[tab1['p_value']<0.05,'Variables'].unique()
print(len(vn1))
vn1

In [ ]:
varList = pd.DataFrame({'Variable': final.columns[1:]})
print(len(varList))
varList['Univariate'] = 0
varList.loc[varList['Variable'].isin(vn1), 'Univariate'] = 1
varList.head()

### Variable Selection using LASSO (L1 penalization)

In [ ]:
from sklearn.linear_model import LogisticRegression
from sklearn.feature_selection import SelectFromModel
lassomod = LogisticRegression(penalty='l1',random_state=0).fit(X,y)
model = SelectFromModel(lassomod, prefit=True)
varList['Lasso'] = model.get_support().astype('int64')

### Variable Selection using Random Forest

In [ ]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.feature_selection import SelectFromModel
rfc=RandomForestClassifier(random_state=10,n_estimators=100,max_depth=2)
rfmod = RandomForestClassifier().fit(X, y)
model = SelectFromModel(rfmod, prefit=True)
varList['RandomForest'] = model.get_support().astype('int64')

### Variable Selection using gradient boosting

In [ ]:
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.feature_selection import SelectFromModel

gbmod = GradientBoostingClassifier(random_state=1000).fit(X, y)
model = SelectFromModel(gbmod, prefit=True)
varList['GradientBoost'] = model.get_support().astype('int64')


### Variable Selection using LinearSVC

In [ ]:
from sklearn.svm import LinearSVC
from sklearn.feature_selection import SelectFromModel

svmmod = LinearSVC(C=0.01, penalty="l1",dual=False,random_state=100).fit(X, y)
model = SelectFromModel(svmmod, prefit=True)
varList['LinearSVC'] = model.get_support().astype('int64')

### Variable Selection using XGB

In [ ]:
random.seed(1000)
import xgboost as xgb
classifier=xgb.XGBClassifier()
finalnum=final.copy()
finalnum.dtypes
columnstoconvert=finalnum.loc[:,finalnum.dtypes=='category'].columns
finalnum[columnstoconvert]=finalnum[columnstoconvert].astype(bool)
Xnum=finalnum.loc[:,finalnum.columns[1:]]
ynum=finalnum['noshowflag']
finalnum.dtypes
xgbmod=classifier.fit(Xnum,ynum)

In [ ]:
from sklearn.feature_selection import SelectFromModel
model = SelectFromModel(xgbmod, prefit=True)
varList['xgb'] = model.get_support().astype('int64')

### Variable Selection using AdaBoost

In [ ]:
from sklearn.ensemble import AdaBoostClassifier
adamod=AdaBoostClassifier(random_state=1000).fit(X,y)
model = SelectFromModel(adamod, prefit=True)
varList['AdaBoost'] = model.get_support().astype('int64')


### Variable Selection using Decision Tree

In [ ]:
from sklearn.tree import DecisionTreeClassifier
treemod=DecisionTreeClassifier(random_state=100).fit(X,y)
model = SelectFromModel(treemod, prefit=True)
varList['DecTree'] = model.get_support().astype('int64')


In [ ]:
if 'Sum' in varList.columns:
    varList=varList.drop(columns=['Sum'])
varList['Sum'] =  np.sum(varList,axis=1)

In [ ]:
selectedfeatures=varList.loc[varList['Sum']>=4,'Variable']
selectedfeatures

In [ ]:
filepath = "C:/Users/moshe/Documents/projectML/p1/data/interim/varList.xlsx"
varList.to_excel(filepath)
filepath = "C:/Users/moshe/Documents/projectML/p1/data/interim/selectedfeatures.csv"
selectedfeatures.to_csv(filepath,index=False,header=True)

In [ ]:
len(selectedfeatures)